In [1]:
import stravalib
import logging
from xml.dom import minidom
import dateutil
import numpy as np
import pandas as pd

import os

from datetime import datetime, timedelta, date


logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
#### Setting up strava API client

#Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client = stravalib.client.Client()
access_token = 'e3ccedc91fceef32077fbb31fc44676446d14bdd'

client.access_token = access_token
athlete = client.get_athlete()

## Enable accessing private activities
auth_url = client.authorization_url(client_id=19435, redirect_uri='http://localhost:8282/authorized', approval_prompt=u'auto', scope='view_private,write', state=None)
from IPython.core.display import display, HTML
display(HTML("""<a href="{}">{}</a>""".format(auth_url,auth_url)))

code = '6d057263b427852b0489af26e921f8fd25a78852'
access_token = client.exchange_code_for_token(client_id=19435, client_secret='45b776d5beceeb34c290b8a56bf9829d6d4ea5d7', code=code)

client = stravalib.client.Client(access_token=access_token)
athlete = client.get_athlete()
print('athlete name %s, athlete id %s.' %(athlete.firstname, athlete.id))

athlete name Brian, athlete id 3360678.


In [3]:
### Setting up google sheets client
import gspread
from oauth2client.service_account import ServiceAccountCredentials


# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
gspread_client = gspread.authorize(creds)
 


In [4]:
## Get last entry from Data Spreadsheet
sheet = gspread_client.open("Milburn Ultimate Scores").sheet1

dates_recorded = [datetime.strptime(d, '%Y-%m-%d') for d in sheet.col_values(1) if d != '' and d != 'Date']
lap_start_date = max(dates_recorded) + timedelta(days=1)

In [5]:
runs = []
for activity in client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        runs.append(activity)

In [6]:
def get_strava_description(activity):
    new_activity = client.get_activity(activity.id)
    try:
        scores, color = new_activity.description.split(' ')
        try:
            team_score, opponent_score = scores.split('-')
        except ValueError:
            team_score, opponent_score, color = None, None, None     
    except ValueError:
        scores = None
        team_score, opponent_score, color = None, None, None           

    return team_score, opponent_score, color.lower()

In [7]:
def extract_events(run):
    lap_nums = []
    start_times = []
    elapsed_times = []
    for l in run.laps:
        lap_nums.append(int(l.name.split(' ')[-1]))
        start_times.append(l.start_date_local)
        elapsed_times.append(l.elapsed_time)

    lap_nums = np.array(lap_nums)
    
    events = []
    for n, s, e in zip(np.diff(lap_nums), start_times, elapsed_times):
        events.append([n, s, e])
    
    return (events)
        
def process_events(events):
    
    event_lookup = [
        '',
        'team_point',
        'opponent_point',
        'my_point',
        'game',
        'game'
    ]
    
    games = []
    game_num = 0
    added = False

    base_game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': 0, 'events':[], 'start_time':None, 'end_time':None}
    game = base_game

    for event in events:
        if game['start_time'] is None:
            game['start_time'] = event[1]

        game['end_time'] = event[1] + event[2]

        event_type = event_lookup[event[0]]
        game['events'].append((event_type, event[1], event[2]))


        if event_type == 'game':
            games.append(game)
            game_num += 1
            game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': game_num, 'events':[], 'start_time':None, 'end_time':None}
            added = True

        elif event_type == 'my_point':
            game[event_type] += 1
            game['team_point'] += 1
            added = False

        else:
            game[event_type] += 1
            added = False


    if not added:
        games.append(game)
    
    
    ## Assign game winners
    for game in games:
        if game['team_point'] > game['opponent_point']:
            game['win'] = True
        else:
            game['win'] = False
            
    return games

In [8]:
games = []
for run in runs:
    team_score, opponent_score, color = get_strava_description(run)
    print(run.start_date_local, get_strava_description(run))
    events = extract_events(run)
    current_games = process_events(events)
    for g in current_games:
        g['my_color'] = color

    games = games + current_games
    

2017-12-18 12:09:17 ('5', '0', 'colors')
2017-12-20 12:28:56 ('1', '2', 'white')
2018-01-03 12:16:29 ('2', '1', 'white')


In [9]:
df = pd.DataFrame(games)
df['date'] = df.end_time.apply(lambda x: date(x.year, x.month, x.day))

df = df.set_index(['date', 'game_num'], drop=False)
df

end_time  \
date       game_num                       
2017-12-18 0        2017-12-18 12:29:15   
           1        2017-12-18 12:53:50   
           2        2017-12-18 13:08:48   
           3        2017-12-18 13:23:36   
2017-12-20 0        2017-12-20 12:47:23   
           1        2017-12-20 13:04:51   
           2        2017-12-20 13:14:34   
2018-01-03 0        2018-01-03 12:47:52   
           1        2018-01-03 13:07:58   

                                                                events  \
date       game_num                                                      
2017-12-18 0         [(team_point, 2017-12-18 12:09:17, 0:03:56), (...   
           1         [(team_point, 2017-12-18 12:29:19, 0:02:23), (...   
           2         [(opponent_point, 2017-12-18 12:54:02, 0:01:48...   
           3         [(opponent_point, 2017-12-18 13:08:52, 0:00:57...   
2017-12-20 0         [(my_point, 2017-12-20 12:28:56, 0:00:31), (op...   
           1         [(team_point, 2017-12-20 12:47:28, 0:01:44), (...   
           2         [(opponent_point, 2017-12-20 13:05:03, 0:04:25...   
2018-01-03 0         [(opponent_point, 2018-01-03 12:16:29, 0:01:17...   
           1         [(opponent_point, 2018-01-03 12:48:15, 0:01:21...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2017-12-18 0                0   colors         3               4   
           1                1   colors         2               5   
           2                2   colors         1               4   
           3                3   colors         3               4   
2017-12-20 0                0    white         3               3   
           1                1    white         0               4   
           2                2    white         0               3   
2018-01-03 0                0    white         3               4   
           1                1    white         2               4   

                             start_time  team_point    win        date  
date       game_num                                                     
2017-12-18 0        2017-12-18 12:09:17           5   True  2017-12-18  
           1        2017-12-18 12:29:19          10   True  2017-12-18  
           2        2017-12-18 12:54:02           5   True  2017-12-18  
           3        2017-12-18 13:08:52           5   True  2017-12-18  
2017-12-20 0        2017-12-20 12:28:56           5   True  2017-12-20  
           1        2017-12-20 12:47:28           5   True  2017-12-20  
           2        2017-12-20 13:05:03           1  False  2017-12-20  
2018-01-03 0        2018-01-03 12:16:29          10   True  2018-01-03  
           1        2018-01-03 12:48:15           5   True  2018-01-03

In [10]:
pdf = df

In [11]:
pdf['white_wins'] = None
pdf['color_wins'] = None
pdf['white_point'] = None
pdf['color_point'] = None
pdf['game_winner'] = None

In [18]:
for (date, game_num), row in pdf.iterrows():      
    
    if 'white' in row.my_color:
        if row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'white_wins'] = team_score                
        pdf.loc[(date, game_num), 'color_wins'] = opponent_score   
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'opponent_point']

    else:
        if not row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'color_wins'] = team_score                
        pdf.loc[(date, game_num), 'white_wins'] = opponent_score   
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'opponent_point']


In [19]:
pdf

end_time  \
date       game_num                       
2017-12-18 0        2017-12-18 12:29:15   
           1        2017-12-18 12:53:50   
           2        2017-12-18 13:08:48   
           3        2017-12-18 13:23:36   
2017-12-20 0        2017-12-20 12:47:23   
           1        2017-12-20 13:04:51   
           2        2017-12-20 13:14:34   
2018-01-03 0        2018-01-03 12:47:52   
           1        2018-01-03 13:07:58   

                                                                events  \
date       game_num                                                      
2017-12-18 0         [(team_point, 2017-12-18 12:09:17, 0:03:56), (...   
           1         [(team_point, 2017-12-18 12:29:19, 0:02:23), (...   
           2         [(opponent_point, 2017-12-18 12:54:02, 0:01:48...   
           3         [(opponent_point, 2017-12-18 13:08:52, 0:00:57...   
2017-12-20 0         [(my_point, 2017-12-20 12:28:56, 0:00:31), (op...   
           1         [(team_point, 2017-12-20 12:47:28, 0:01:44), (...   
           2         [(opponent_point, 2017-12-20 13:05:03, 0:04:25...   
2018-01-03 0         [(opponent_point, 2018-01-03 12:16:29, 0:01:17...   
           1         [(opponent_point, 2018-01-03 12:48:15, 0:01:21...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2017-12-18 0                0   colors         3               4   
           1                1   colors         2               5   
           2                2   colors         1               4   
           3                3   colors         3               4   
2017-12-20 0                0    white         3               3   
           1                1    white         0               4   
           2                2    white         0               3   
2018-01-03 0                0    white         3               4   
           1                1    white         2               4   

                             start_time  team_point    win        date  \
date       game_num                                                      
2017-12-18 0        2017-12-18 12:09:17           5   True  2017-12-18   
           1        2017-12-18 12:29:19          10   True  2017-12-18   
           2        2017-12-18 12:54:02           5   True  2017-12-18   
           3        2017-12-18 13:08:52           5   True  2017-12-18   
2017-12-20 0        2017-12-20 12:28:56           5   True  2017-12-20   
           1        2017-12-20 12:47:28           5   True  2017-12-20   
           2        2017-12-20 13:05:03           1  False  2017-12-20   
2018-01-03 0        2018-01-03 12:16:29          10   True  2018-01-03   
           1        2018-01-03 12:48:15           5   True  2018-01-03   

                    white_wins color_wins white_point color_point game_winner  
date       game_num                                                            
2017-12-18 0                 1          2           4           5       Color  
           1                 1          2           5          10       Color  
           2                 1          2           4           5       Color  
           3                 1          2           4           5       Color  
2017-12-20 0                 2          1           5           3       White  
           1                 2          1           5           4       White  
           2                 2          1           1           3       Color  
2018-01-03 0                 2          1          10           4       White  
           1                 2          1           5           4       White

In [21]:
pdf[['white_wins', 'color_wins', 'game_num', 'game_winner']].as_matrix()

array([['1', '2', 0, 'Color'],
       ['1', '2', 1, 'Color'],
       ['1', '2', 2, 'Color'],
       ['1', '2', 3, 'Color'],
       ['2', '1', 0, 'White'],
       ['2', '1', 1, 'White'],
       ['2', '1', 2, 'Color'],
       ['2', '1', 0, 'White'],
       ['2', '1', 1, 'White']], dtype=object)